<a href="https://colab.research.google.com/github/AfrizalSY/ML_KMeans_Clustering/blob/main/K_Means_Clustering_Machine_Learning_1301194288.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Afrizal Syahruluddin Yusuf

NIM  : 1301194288

Kelas: IF-43-09

In [ ]:
!pip install missingno

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

from scipy import stats
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

**Pemanggilan Data**

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/AfrizalSY/ML_KMeans_Clustering/main/kendaraan_train.csv")

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
msno.matrix(data)

In [ ]:
msno.bar(data)

In [ ]:
data.isna().sum()*100/len(data)

In [ ]:
df = data

In [ ]:
df

In [ ]:
df['Umur']=df['Umur'].fillna(df['Umur'].mean())
df['Premi']=df['Premi'].fillna(df['Kanal_Penjualan'].mean())
df['Kanal_Penjualan']=df['Kanal_Penjualan'].fillna(df['Kanal_Penjualan'].mean())
df['Lama_Berlangganan']=df['Lama_Berlangganan'].fillna(df['Lama_Berlangganan'].mean())

In [ ]:
df['Umur_Kendaraan']=df['Umur_Kendaraan'].fillna(df['Umur_Kendaraan'].mode().iloc[0])
df['Kode_Daerah']=df['Kode_Daerah'].fillna(df['Kode_Daerah'].mode().iloc[0])
df['Kendaraan_Rusak']=df['Kendaraan_Rusak'].fillna(df['Kendaraan_Rusak'].mode().iloc[0])
df['Jenis_Kelamin']=df['Jenis_Kelamin'].fillna(df['Jenis_Kelamin'].mode().iloc[0])
df['SIM']=df['SIM'].fillna(df['SIM'].mode().iloc[0])
df['Sudah_Asuransi']=df['Sudah_Asuransi'].fillna(df['Sudah_Asuransi'].mode().iloc[0])

In [ ]:
dummies = pd.get_dummies(df[['Jenis_Kelamin', 'Kendaraan_Rusak']], drop_first=True)
df = pd.concat([df.drop(['Jenis_Kelamin', 'Kendaraan_Rusak'],axis=1), dummies],axis=1)

dummies2 = pd.get_dummies(df['Umur_Kendaraan'])
df = pd.concat([df.drop(['Umur_Kendaraan'],axis=1),dummies2],axis=1)

In [ ]:
dummies = pd.get_dummies(df['SIM'],drop_first=True)
df=df.drop(['SIM'],axis=1)
df['SIM_Ada'] = dummies[1.0]

dummies = pd.get_dummies(df['Sudah_Asuransi'],drop_first=True)
df=df.drop(['Sudah_Asuransi'],axis=1)
df['Sudah_Asuransi'] = dummies[1.0]

In [ ]:
df.isna().any()

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.corr().style.background_gradient()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(df.corr(), annot=True, linewidth=1, ax=ax)

In [ ]:
df1 = df[['Umur', 'Kanal_Penjualan', 'Tertarik']]

In [ ]:
plt.scatter(df1['Umur'], df1['Kanal_Penjualan'])

In [ ]:
minmax = MinMaxScaler()
minmax_data = minmax.fit_transform(df1)
plt.scatter(minmax_data[:,0], minmax_data[:,1])

In [ ]:
pca = PCA(2)
pca_data = pca.fit_transform(minmax_data)
plt.scatter(pca_data[:,0], pca_data[:,1], c = 'lightblue' )

In [ ]:
box = df1.boxplot(['Kanal_Penjualan', 'Umur'])

In [ ]:
print(round(df1.describe(), 2))

In [ ]:
def kmeans(x,k, iter):
    idx = np.random.choice(len(x), k, replace=False)
    #Random Centroid
    centroids = x[idx, :]
    #Menentukan jarak antara centroid dan semua data
    distances = cdist(x, centroids, 'euclidean')
    #Centroid dengan jarak paling dekat/kecil
    points = np.array([np.argmin(i) for i in distances])

    for _ in range(iter):
      centroids = []
      for idx in range(k):
          #Update lokasi centroid
          temp_cent = x[points==idx].mean(axis=0)
          centroids.append(temp_cent)
      centroids = np.vstack(centroids)
      distances = cdist(x, centroids, 'euclidean')
      points = np.array([np.argmin(i) for i in distances])
    return points, centroids

#pca = PCA(2)
#data = pca.fit_transform(dfg)
result = kmeans(pca_data,2, 10)
label = result[0]
centroid = result[1]

In [ ]:
label

In [ ]:
u_labels = np.unique(label)
for i in u_labels:
    for k in range(len(centroid)):
      plt.scatter(centroid[k][0],centroid[k][1], zorder=2, color="black")
    plt.scatter(pca_data[label == i , 0] , pca_data[label == i , 1] , alpha=.5, label = i)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(26,10))
sns.countplot(x="Umur", data=data,ax=ax)

In [ ]:
dd2 = df1
dd2['Clusters'] = label
dd2 = dd2[['Umur', 'Kanal_Penjualan', 'Tertarik', 'Clusters']]
dd2.groupby(['Clusters', 'Tertarik']).count()

In [ ]:
data_akhir=data
data_akhir['Clusters']=label
data_akhir.to_csv(r'kendaraan_fix.csv')

In [ ]:
data_fix=pd.read_csv('kendaraan_fix.csv')

In [ ]:
data_fix[data_fix['Clusters']==1]